In [1]:
import os
import pickle
import numpy as np
import pandas as pd

from glob import glob

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, random_split
from torchvision import models, transforms

from PIL import Image

In [2]:
transform_resnet = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

transform_cifar = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

In [3]:
class CIFAR10Dataset(Dataset):
    def __init__(self, data_path="cifar-10-batches-py", train=True, transform=None):
        self.transform = transform
        self.data = []
        self.labels = []
        if train:
            for i in range(1, 6):
                with open(os.path.join(data_path, f"data_batch_{i}"), "rb") as f:
                    batch = pickle.load(f, encoding='bytes')
                    self.data.append(batch[b'data'])
                    self.labels.extend(batch[b'labels'])
            self.data = np.vstack(self.data).astype(np.uint8)
        else:
            with open(os.path.join(data_path, "test_batch"), "rb") as f:
                batch = pickle.load(f, encoding='bytes')
                self.data = batch[b'data'].astype(np.uint8)
                self.labels = batch[b'labels']
        # convert to HWC
        self.data = self.data.reshape(-1, 3, 32, 32).transpose(0, 2, 3, 1)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        img = self.data[idx]
        label = self.labels[idx]
        img = Image.fromarray(img)
        if self.transform:
            img = self.transform(img)
        return img, label


def load_cifar10_resnet(data_path="cifar-10-batches-py"):
    full_train_dataset = CIFAR10Dataset(
        data_path=data_path, train=True, transform=transform_resnet)
    train_size = int(0.8 * len(full_train_dataset))
    val_size = len(full_train_dataset) - train_size
    train_dataset, val_dataset = random_split(
        full_train_dataset, [train_size, val_size])
    test_dataset = CIFAR10Dataset(
        data_path=data_path, train=False, transform=transform_resnet)
    return train_dataset, val_dataset, test_dataset


def load_cats_dogs_resnet(folder_path="dogs-vs-cats/train"):
    class CatsDogsDataset(Dataset):
        def __init__(self, folder_path, transform=None):
            self.transform = transform
            self.image_paths = []
            self.labels = []
            for path in glob(os.path.join(folder_path, "*.jpg")):
                self.image_paths.append(path)
                fname = os.path.basename(path).lower()
                if "cat" in fname:
                    self.labels.append(0)
                elif "dog" in fname:
                    self.labels.append(1)
                else:
                    raise ValueError(f"Unknown label in file name: {fname}")

        def __len__(self):
            return len(self.labels)

        def __getitem__(self, idx):
            img = Image.open(self.image_paths[idx]).convert("RGB")
            label = self.labels[idx]
            if self.transform:
                img = self.transform(img)
            return img, label

    full_dataset = CatsDogsDataset(folder_path, transform=transform_resnet)
    train_size = int(0.8 * len(full_dataset))
    val_size = len(full_dataset) - train_size
    train_dataset, val_dataset = random_split(
        full_dataset, [train_size, val_size])
    return train_dataset, val_dataset

In [4]:
class CustomCNN(nn.Module):
    def __init__(self, num_classes=10, activation="relu", input_shape=(3, 32, 32)):
        super().__init__()

        act = activation.lower()
        if act == "relu":
            self.act = nn.ReLU()
        elif act == "tanh":
            self.act = nn.Tanh()
        elif act == "leaky_relu":
            self.act = nn.LeakyReLU()
        else:
            raise ValueError("Invalid activation")

        self.conv1 = nn.Conv2d(input_shape[0], 32, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.pool = nn.MaxPool2d(2, 2)

        self.fc1 = nn.Linear(
            128 * (input_shape[1] // 8) * (input_shape[2] // 8), 256)
        self.drop = nn.Dropout(0.5)
        self.fc2 = nn.Linear(256, num_classes)

    def forward(self, x):
        x = self.pool(self.act(self.bn1(self.conv1(x))))
        x = self.pool(self.act(self.bn2(self.conv2(x))))
        x = self.pool(self.act(self.bn3(self.conv3(x))))
        x = x.view(x.size(0), -1)
        x = self.drop(self.act(self.fc1(x)))
        x = self.fc2(x)
        return x

In [5]:
def build_resnet18(num_classes, input_size=(224, 224)):
    model = models.resnet18(weights=None)
    if model.conv1.in_channels != 3:
        model.conv1 = nn.Conv2d(3, 64, kernel_size=7,
                                stride=2, padding=3, bias=False)
    in_features = model.fc.in_features
    model.fc = nn.Linear(in_features, num_classes)
    return model


def train_resnet(model, dataname, train_loader, val_loader, optimizer, criterion, epochs=10, device='cpu'):
    model.to(device)
    best_val_acc = 0.0

    for epoch in range(epochs):
        model.train()
        running_loss = 0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)

        avg_loss = running_loss / len(train_loader.dataset)

        model.eval()
        correct, total = 0, 0
        val_loss = 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item() * inputs.size(0)
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        val_acc = 100 * correct / total
        print(f"Epoch {epoch+1}/{epochs}, Train Loss: {avg_loss:.4f}, Val Loss: {val_loss/len(val_loader.dataset):.4f}, Val Acc: {val_acc:.2f}%")

        if val_acc > best_val_acc:
            best_val_acc = val_acc
            os.makedirs(f"resnet_models/{dataname}", exist_ok=True)
            torch.save(model.state_dict(),
                       f"resnet_models/{dataname}/resnet18_best.pth")

    print("Training complete. Best Val Accuracy:", best_val_acc)

In [6]:
def evaluate_model(model, val_loader, criterion, device='cpu'):
    model.to(device)
    model.eval()
    correct, total = 0, 0
    running_loss = 0

    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            running_loss += loss.item() * inputs.size(0)

            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    avg_loss = running_loss / total
    return accuracy, avg_loss

In [7]:
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"

cifar_train, cifar_val, cifar_test = load_cifar10_resnet()
cifar_trainloader = DataLoader(cifar_train, batch_size=64, shuffle=True)
cifar_valloader = DataLoader(cifar_val, batch_size=64, shuffle=False)

model_cifar = build_resnet18(num_classes=10)
optimizer = optim.Adam(model_cifar.parameters(), lr=1e-4)
criterion = nn.CrossEntropyLoss()

train_resnet(model_cifar, "cifar", cifar_trainloader,
             cifar_valloader, optimizer, criterion, epochs=10, device=device)

dvc_train, dvc_val = load_cats_dogs_resnet()
dvc_trainloader = DataLoader(dvc_train, batch_size=32, shuffle=True)
dvc_valloader = DataLoader(dvc_val, batch_size=32, shuffle=False)

model_dvc = build_resnet18(num_classes=2)
optimizer = optim.Adam(model_dvc.parameters(), lr=1e-4)
criterion = nn.CrossEntropyLoss()

train_resnet(model_dvc, "dvc", dvc_trainloader, dvc_valloader,
             optimizer, criterion, epochs=10, device=device)

/var/folders/mz/34p9h3ld08ldtfp6qbl62tmc0000gn/T/ipykernel_45658/1812573342.py:9: VisibleDeprecationWarning: dtype(): align should be passed as Python or NumPy boolean but got `align=0`. Did you mean to pass a tuple to create a subarray type? (Deprecated NumPy 2.4)
  batch = pickle.load(f, encoding='bytes')
/var/folders/mz/34p9h3ld08ldtfp6qbl62tmc0000gn/T/ipykernel_45658/1812573342.py:15: VisibleDeprecationWarning: dtype(): align should be passed as Python or NumPy boolean but got `align=0`. Did you mean to pass a tuple to create a subarray type? (Deprecated NumPy 2.4)
  batch = pickle.load(f, encoding='bytes')


Epoch 1/10, Train Loss: 1.3414, Val Loss: 1.1672, Val Acc: 59.29%
Epoch 2/10, Train Loss: 0.9058, Val Loss: 0.9468, Val Acc: 66.90%
Epoch 3/10, Train Loss: 0.6853, Val Loss: 0.8507, Val Acc: 71.48%
Epoch 4/10, Train Loss: 0.5081, Val Loss: 0.7650, Val Acc: 73.51%
Epoch 5/10, Train Loss: 0.3389, Val Loss: 0.9067, Val Acc: 71.15%
Epoch 6/10, Train Loss: 0.2014, Val Loss: 0.9584, Val Acc: 72.77%
Epoch 7/10, Train Loss: 0.1146, Val Loss: 0.9036, Val Acc: 74.18%
Epoch 8/10, Train Loss: 0.0775, Val Loss: 0.9485, Val Acc: 73.82%
Epoch 9/10, Train Loss: 0.0701, Val Loss: 1.0792, Val Acc: 72.28%
Epoch 10/10, Train Loss: 0.0618, Val Loss: 1.0741, Val Acc: 73.50%
Training complete. Best Val Accuracy: 74.18
Epoch 1/10, Train Loss: 0.5482, Val Loss: 0.4396, Val Acc: 79.44%
Epoch 2/10, Train Loss: 0.3881, Val Loss: 0.3817, Val Acc: 82.04%
Epoch 3/10, Train Loss: 0.2794, Val Loss: 0.2527, Val Acc: 89.32%
Epoch 4/10, Train Loss: 0.1943, Val Loss: 0.2450, Val Acc: 89.22%
Epoch 5/10, Train Loss: 0.1251,

In [10]:
df = pd.DataFrame(pd.read_csv('experiment_results.csv'))
best_cifar = df[df.dataset == "Cifar-10"].nlargest(1, "accuracy")
best_dvc = df[df.dataset == "Dogs vs Cats"].nlargest(1, "accuracy")

best_cifar_model_path = f"models/cifar/model_{best_cifar.iloc[0]['activation']}_{best_cifar.iloc[0]['init']}_{best_cifar.iloc[0]['optimizer']}_best.pth"
best_dvc_model_path = f"models/dvc/model_{best_dvc.iloc[0]['activation']}_{best_dvc.iloc[0]['init']}_{best_dvc.iloc[0]['optimizer']}_best.pth"

print("Best CIFAR-10 model path:", best_cifar_model_path)
print("Best Dogs vs Cats model path:", best_dvc_model_path)

Best CIFAR-10 model path: models/cifar/model_leaky_relu_random_rmsprop_best.pth
Best Dogs vs Cats model path: models/dvc/model_leaky_relu_kaiming_adam_best.pth


In [17]:
criterion = nn.CrossEntropyLoss()
results = []

resnet_model = build_resnet18(num_classes=10)
resnet_model.load_state_dict(torch.load(
    "resnet_models/cifar/resnet18_best.pth", map_location=device))
resnet_acc, resnet_loss = evaluate_model(
    resnet_model, cifar_valloader, criterion, device)
results.append({
    'dataset': 'CIFAR-10',
    'model': 'ResNet-18',
    'accuracy': resnet_acc,
    'loss': resnet_loss
})

results.append({
    'dataset': 'CIFAR-10',
    'model': f"{best_cifar.iloc[0]['activation']}_{best_cifar.iloc[0]['init']}_{best_cifar.iloc[0]['optimizer']}",
    'accuracy': best_cifar.iloc[0]['accuracy'],
    'loss': best_cifar.iloc[0]['val_loss']
})

resnet_model = build_resnet18(num_classes=2)
resnet_model.load_state_dict(torch.load(
    "resnet_models/dvc/resnet18_best.pth", map_location=device))
resnet_acc, resnet_loss = evaluate_model(
    resnet_model, dvc_valloader, criterion, device)
results.append({
    'dataset': 'Dogs vs Cats',
    'model': 'ResNet-18',
    'accuracy': resnet_acc,
    'loss': resnet_loss
})

results.append({
    'dataset': 'Dogs vs Cats',
    'model': f"{best_dvc.iloc[0]['activation']}_{best_dvc.iloc[0]['init']}_{best_dvc.iloc[0]['optimizer']}",
    'accuracy': best_dvc.iloc[0]['accuracy'],
    'loss': best_dvc.iloc[0]['val_loss']
})

df = pd.DataFrame(results)
df.to_csv("comparison_results.csv", index=False)
print("Comparison saved to comparison_results.csv")


Comparison saved to comparison_results.csv


In [18]:
df = pd.DataFrame(pd.read_csv('comparison_results.csv'))
df

,dataset,model,accuracy,loss
0,CIFAR-10,ResNet-18,74.18,0.903581
1,CIFAR-10,leaky_relu_random_rmsprop,84.03,0.463258
2,Dogs vs Cats,ResNet-18,90.22,0.255094
3,Dogs vs Cats,leaky_relu_kaiming_adam,89.74,0.234952
